In [2]:
#import fitz  
#from PIL import Image
#import pytesseract
import io
import re
import os
#pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
#import easyocr
import numpy as np
from dotenv import load_dotenv
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import ClientAuthenticationError, HttpResponseError
load_dotenv()


ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
endpoint = os.getenv("AZURE_FORM_RECOGNIZER_ENDPOINT")
key = os.getenv("AZURE_FORM_RECOGNIZER_KEY")
endpoint="https://group3documentintelligence.cognitiveservices.azure.com/"
key = "10sJZXvhWvbBSfZ5NbJ4Mn3zJ7dJLBPcxH2cneXTub9ChN9MWGzUJQQJ99BEAC5RqLJXJ3w3AAALACOGL4P1"

client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

with open("Resource/Übersicht3.pdf", "rb") as f:
    poller = client.begin_analyze_document("prebuilt-layout", document=f)
    result = poller.result()

for i, page in enumerate(result.pages, start=1):
    lines = [line.content for line in page.lines]
    page_text = " ".join(lines)
    output_path = f"Text/page_{i}.txt"
    with open(output_path, "w", encoding="utf-8") as out_file:
        out_file.write(page_text)

    print(f"✅ Page {i} saved to {output_path}")


In [99]:
def primera_posicion_espacio(texto):
    return texto.find(' ')  # devuelve índice del primer espacio, o -1 si no hay

def number_posicion_numero(texto,number):
    count = 0
    for i, c in enumerate(texto):
        if c.isdigit():
            count += 1
            if count == number:
                return i
    return -1  # si no hay un segundo número, devuelve -1

index = set()
texto1 = "Technische Angaben Schraubenverdichteraggregat FH-5GC Übersicht 1. UMGEBUNGSBEDINGUNGEN 2 2. PROJEKTIERTE BETRIEBSBEDINGUNGEN 2 3. HÖCHSTZULÄSSIGE BETRIEBSDRÜCKE 2 4. EINSTELLWERTE FÜR SICHERHEITSVENTILE 2 5. EINSTELLWERTE DER SICHERHEITSGERÄTE GEGEN UNZULÄSSIGE DRÜCKE 3 6. EINSTELLWERTE DER SICHERHEITSGERÄTE GEGEN Kältemittelkreislauf 23 bar Überdruck UNZULÄSSIGE TEMPERATUREN 3 Ölkreislauf 23 bar Überdruck 7. SONSTIGE EINSTELLWERTE VON SCHALTERN UND ARMATUREN 3 8. RICHTWERTE BETRIEB DES VERDICHTERAGGREGATES 3 9. HAUPTBAUTEILE 10. SENSOREN 4 AFD/ Sche. (Ausgabe/ edition 02.05.96) GRassa KAB Refrigeration Equipment 1. Umgebungsbedingungen · Umgebungstemperatur max.40 ℃ min.5 ℃ · frei von Flüssigkeitsniederschlägen 2. Projektierte Betriebsbedingungen Sättigungstemperatur Verdichtersaugdruck to -5 ℃ +5 ℃ Überhitzung im Verdichtersaugstutzen 10 K 10 K Sättigungstemperatur Verdichtungsenddruck tx +27 °℃ 30 °℃ Aufladung des Verdichters nein nein 3. Höchstzulässige Betriebsdrücke 4. Einstellwerte für Sicherheitsventile Sicherheitsventil (3.1) Ölabscheider DN 15 Überströmventil (1.1) (von Verdichterdruck- zur -saugseite) DN 20 4 23 bar Überdruck 20 bar Überdruck Seite: von Page: of 2 4 631726.DOC Revision 0 KÜHLAUTOMAT Berlin GmbH KAB"
texto2 = "Technische Angaben Schraubenverdichteraggregat FH-5GC 5. Einstellwerte der Sicherheitsgeräte gegen unzulässige Drücke Druckschalter (15.8) Verdichtungsenddruck Abschaltdruck 15 bar Überdruck Saugdruckschalter (15.7) Abschaltdruck 1,5 bar Überdruck Differenzdruckwächter (15.9) Ölkreislaufüberwachung Abschaltdifferenzdruck ≤ 0,5 bar Differenzdruckwächter (15.10) Ölgrobfilterüberwachung Abschaltdifferenzdruck ≥ 2 bar 6. Einstellwerte der Sicherheitsgeräte gegen unzulässige Temperaturen Pt 100 (17.2) für Verdichtungsendtemperatur Schaltwert 90 °℃ Pt 100 (17.3) für Öltemperatur Schaltwert 60 ℃ Temperaturbegrenzer (3.10) Heizkörpertemperatur Schaltwert 150°℃ Thermostat (3.9) Heizkörpertemperatur Einstellwert 70°C 7. Sonstige Einstellwerte von Schaltern und Armaturen Öldruckregelventil (8.0) 4,5+0,5 bar Absperrventil (18.2) in Gasschwingungsschutzleitung geschlossen AFD/ Sche. (Ausgabe/ edition 02.05.96) GRassaiKAB Refrigeration Equipment 8. Richtwerte Betrieb des Verdichteraggregates Mindestverstellzeit des Regelschlittens von Min- zur Max-Endlage bzw. umgekehrt 30s Anzustrebende Verstellzeit des Regelschlittens von Min- zur Max-Endlage bzw. umgekehrt 60s Bei Anfahrvorgängen und kaltem Öl und bei niedrigen Druckdifferenzen zwischem dem Verdichtungsenddruck und Saugdruck sind die Verstellzeiten höher als bei Betriebsbedingungen. 8.1. Einstellwerte für Verdichtungsendtemperatur anzustrebender Wert t 80°℃ maximaler Wert tmax 90°C 8.2. Öltemperatur zulässiger Bereich der Öltemperatur 35 - 55°C töl anzustrebende Öltemperatur töl 45°℃ 8.3. Ölfüllung Erstfüllung ca. 110 1 8.4. Hilfs- und Steuerspannungen Magnetventile für 230 V,AC Leistungsregelung und Vi.Verst. Überwachungsgeräte, Druck und Temperatur, Füllstand 24 V,DC Wegsensor für Regelschlitten 24 V,DC Ölheizung vom Kraftstromfeld 2 × 440 V, AC 0. 2 × 220 V, AC Seite: 3 von Page: of 4 631726.DOC Revision 0 KÜHLAUTOMAT Berlin GmbH KAB"
texto3 = "Technische Angaben Schraubenverdichteraggregat FH-5GC 9. Hauptbauteile Bezeichnung Typ Verdichter F-5 Leistungsregelung MV-Block S 25 Antriebsmotor P 200 L, 66 KW Beistellung Kupplung 66 E 60 Dm 60 Ölpumpe LG 0/10 Ölheizung 380-380-1200 (2x440V o. 2×220V) Ölabscheider Dm 508 (3) TÜV Ölkühler 12/1100/35KW Saugfilter 80/100-40.1 Ölfilter FE140×34 / 459 NH3 Steuerung GRENCO B.V. Software GRENCO B.V. AFD/ Sche. (Ausgabe/ edition 02.05.96) Refrigeration Equipment 10.Sensoren Bezeichnung Typ Saugdruckstörung RT 1A Saugdruckregelung AKS 33 Enddruckstörung RT 6AW Enddruck AKS 33 Öldifferenzdruck Ölkreislaufüber- wachung RT 260 A Öldifferenzdruck Filterüberwachung RT 260 A Öldruck nach Ölpumpe AKS 33 Ansaugtemperatur Pt100 Endtemperatur Pt100 Öltemperatur Pt100 Öltemperatur- regelung Pt100 Wegsensor Regelschlitten HDT Motorstrom Wandler und Meßumformer im Schaltschrank 631726.DOC Revision 0 Seite: von Page: of 4 4 KÜHLAUTOMAT Berlin GmbH KAB"

def index_from_text(texto):
    while len(texto)>0:
        first_position_number = number_posicion_numero(texto,1)
        
        if texto[first_position_number+1] == "." and texto[first_position_number+2] == " " and texto[first_position_number + 3].isalpha():
            
            second_position_number = number_posicion_numero(texto,2)
            first_index = texto[first_position_number:second_position_number]
            index.add(first_index)
            texto = texto[second_position_number+1:]
        
        # elif texto[first_position_number+1] == "." and texto[first_position_number+2].isdigit():
        
        #     third_position_number = number_posicion_numero(texto,3)
        #     first_index = texto[first_position_number:third_position_number]
        #     print(first_index)
        #     texto = texto[third_position_number+1:]
        
        else: 
            texto = texto[first_position_number+1:]
            if first_position_number == -1:
                break

index_from_text(texto1)
index_from_text(texto2)
index_from_text(texto3)
print(index)

{'5. EINSTELLWERTE DER SICHERHEITSGERÄTE GEGEN UNZULÄSSIGE DRÜCKE ', '3. Höchstzulässige Betriebsdrücke ', '6. Einstellwerte der Sicherheitsgeräte gegen unzulässige Temperaturen Pt ', '9. HAUPTBAUTEILE ', '4. Hilfs- und Steuerspannungen Magnetventile für ', '1. Umgebungsbedingungen · Umgebungstemperatur max.', '5. Einstellwerte der Sicherheitsgeräte gegen unzulässige Drücke Druckschalter (', '2. PROJEKTIERTE BETRIEBSBEDINGUNGEN ', '8. Richtwerte Betrieb des Verdichteraggregates Mindestverstellzeit des Regelschlittens von Min- zur Max-Endlage bzw. umgekehrt ', '2. Projektierte Betriebsbedingungen Sättigungstemperatur Verdichtersaugdruck to -', '8. RICHTWERTE BETRIEB DES VERDICHTERAGGREGATES ', '1. UMGEBUNGSBEDINGUNGEN ', '9. Hauptbauteile Bezeichnung Typ Verdichter F-', '4. EINSTELLWERTE FÜR SICHERHEITSVENTILE ', '6. EINSTELLWERTE DER SICHERHEITSGERÄTE GEGEN Kältemittelkreislauf ', '3. HÖCHSTZULÄSSIGE BETRIEBSDRÜCKE ', '0. SENSOREN ', '2. Öltemperatur zulässiger Bereich der Öltemperatur

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Choose the instruct model variant (e.g., 4k context version)
model_id = "microsoft/phi-3-mini-4k-instruct"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# Prompt
prompt = "Explain what a black hole is in simple terms."

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.7)

# Decode output
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)




AttributeError: partially initialized module 'torch' has no attribute '_subclasses' (most likely due to a circular import)

In [4]:
from Jiahao.Helper import sort_toc_lines
from Jiahao.Helper import separate_on_number
#from Jiahao.Helper import extract_toc_pages
from Jiahao.Helper import is_valid_toc_line


In [28]:
def extract_toc_pages(pdf_path, lang='deu'):
    doc = fitz.open(pdf_path)  # Open PDF file
    toc_pages = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes()))
        text = pytesseract.image_to_string(img, lang=lang)
         
        if "übersicht" in text.lower():
        #if "inhaltsverzeichnis" in text.lower() or "übersicht" in text.lower():
            return text, img 

In [ ]:
def extract_toc_pages(pdf_path, lang='de'):
    doc = fitz.open(pdf_path)
    reader = easyocr.Reader([lang])  # Initialize EasyOCR with German language
    toc_pages = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes()))
        
        # Convert PIL Image to numpy array for EasyOCR
        img_np = np.array(img)
        
        # Run OCR
        result = reader.readtext(img_np, detail=0)  # detail=0 to get only text strings
        
        full_text = "\n".join(result)
        
        if "übersicht" in full_text.lower():
            return full_text

    return None, None  # If no TOC page is found


In [ ]:
text= extract_toc_pages("Resource/Übersicht3.pdf")

print(text)


process_text = separate_on_number(text)
print(process_text)


valid_lines = [line for line in process_text if is_valid_toc_line(line)]
# for i in valid_lines:
#     print(i)

sorted_lines = sort_toc_lines(valid_lines)
# for i in sorted_lines:
#     print(i)



Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\j7843\Documents\DAIA\GEA_llm_pdf_processing_group17\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Technische Angaben
Grass6
KAB
Schraubenverdichteraggregat
Actriocreticn Eqciamcii
FH-SGC
Übersicht
1.
Umgebungsbedingungen
Umgebungstemperatur
max.40 'C
min.5 'C
1. UMGEBUNGSBEDINGUNGEN
2
frei von Flüssigkeitsniederschlägen
2. PROJEKTIERTE
BETRIEBSBEDINGUNGEN
2
2. Projektierte
Betriebsbedingungen
3. HOCHSTZULÄSSIGE
Sättigungstemperatur
~5 *C
+5 'C
BETRIEBSDRÜCKE
Verdichtersaugdruck to
Überhitzung im
10 K
10 K
4. EINSTELLWERTE FÜR
Verdichtersaugstutzen
SICHERHEITSVENTILE
2
Sättigungstemperatur
+27 'C
30 'C
Verdichtungsenddruck tK
5. EINSTELLWERTE DER
Aufladung des Verdichters
nein
nein
SICHERHEITSGERÄTE GEGEN
UNZULÄSSIGE DRÜCKE
3.
Hochstzulässige Betriebsdrücke
6. EINSTELLWERTE DER
SICHERHEITSGERÄTE GEGEN
Kältemittelkreislauf
23 bar Überdruck
UNZULÄSSIGE TEMPERATUREN
3
Ölkreislauf
23 bar Überdruck
7. SONSTIGE EINSTELLWERTE VON
SCHALTERN UND ARMATUREN
4. Einstellwerte für Sicherheitsventile
8. RICHTWERTE BETRIEB DES
Sicherheitsventil (3.1)
23 bar Überdruck
VERDICHTERAGGREGATES
Ölabscheid

In [43]:
import re

def extract_numbered_sections(text):
    """
    Extract sections starting with numbers followed by dots and connect them to the following words
    until a non-letter/non-space character is found.

    Args:
        text (str): The full OCR text string.

    Returns:
        list of tuples: Each tuple is (number_prefix, connected_text)
    """
    pattern = re.compile(r"(\d+(?:\.\d+)*\.)\s")  # Match number-dot with a space after it
    
    results = []
    pos = 0
    length = len(text)
    
    while pos < length:
        match = pattern.search(text, pos)
        if not match:
            break
        
        number_prefix = match.group(1)  # e.g. '1.' or '1.2.'
        start = match.end()  # position after the matched number and space
        
        # Collect all words following the number prefix until hitting a non-letter, non-space char
        connected_words = []
        i = start
        while i < length:
            c = text[i]
            if c.isalpha() or c.isspace():
                connected_words.append(c)
                i += 1
            else:
                break
        
        connected_text = "".join(connected_words).strip()
        results.append((number_prefix, connected_text))
        
        pos = i  # continue search after the collected section
    
    return results

extract = extract_numbered_sections(text)
print(extract)

[('1.', 'Umgebungsbedingungen\nUmgebungstemperatur\nmax'), ('1.', 'UMGEBUNGSBEDINGUNGEN'), ('2.', 'PROJEKTIERTE\nBETRIEBSBEDINGUNGEN'), ('2.', 'Projektierte\nBetriebsbedingungen'), ('3.', 'HOCHSTZULÄSSIGE\nSättigungstemperatur'), ('4.', 'EINSTELLWERTE FÜR\nVerdichtersaugstutzen\nSICHERHEITSVENTILE'), ('5.', 'EINSTELLWERTE DER\nAufladung des Verdichters\nnein\nnein\nSICHERHEITSGERÄTE GEGEN\nUNZULÄSSIGE DRÜCKE'), ('3.', 'Hochstzulässige Betriebsdrücke'), ('6.', 'EINSTELLWERTE DER\nSICHERHEITSGERÄTE GEGEN\nKältemittelkreislauf'), ('7.', 'SONSTIGE EINSTELLWERTE VON\nSCHALTERN UND ARMATUREN'), ('4.', 'Einstellwerte für Sicherheitsventile'), ('8.', 'RICHTWERTE BETRIEB DES\nSicherheitsventil'), ('9.', 'HAUPTBAUTEILE\nzur'), ('10.', 'SENSOREN')]


In [42]:
process_text = separate_on_number(text)
print(process_text)

['Technische Angaben', 'Grass6', 'KAB', 'Schraubenverdichteraggregat', 'Actriocreticn Eqciamcii', 'FH-SGC', 'Übersicht', '1.', 'Umgebungsbedingungen', 'Umgebungstemperatur', "max.40 'C", "min.5 'C", '1. UMGEBUNGSBEDINGUNGEN', '2', 'frei von Flüssigkeitsniederschlägen', '2. PROJEKTIERTE', 'BETRIEBSBEDINGUNGEN', '2', '2. Projektierte', 'Betriebsbedingungen', '3. HOCHSTZULÄSSIGE', 'Sättigungstemperatur', '~5 *C', "+5 'C", 'BETRIEBSDRÜCKE', 'Verdichtersaugdruck to', 'Überhitzung im', '10 K', '10 K', '4. EINSTELLWERTE FÜR', 'Verdichtersaugstutzen', 'SICHERHEITSVENTILE', '2', 'Sättigungstemperatur', "+27 'C", "30 'C", 'Verdichtungsenddruck tK', '5. EINSTELLWERTE DER', 'Aufladung des Verdichters', 'nein', 'nein', 'SICHERHEITSGERÄTE GEGEN', 'UNZULÄSSIGE DRÜCKE', '3.', 'Hochstzulässige Betriebsdrücke', '6. EINSTELLWERTE DER', 'SICHERHEITSGERÄTE GEGEN', 'Kältemittelkreislauf', '23 bar Überdruck', 'UNZULÄSSIGE TEMPERATUREN', '3', 'Ölkreislauf', '23 bar Überdruck', '7. SONSTIGE EINSTELLWERTE VON',

In [41]:
sorted_lines = sort_toc_lines(valid_lines)
for i in sorted_lines:
    print(i)

1. UMGEBUNGSBEDINGUNGEN
2. PROJEKTIERTE
2. Projektierte
3. HOCHSTZULÄSSIGE
4. EINSTELLWERTE FÜR
4. Einstellwerte für Sicherheitsventile
5. EINSTELLWERTE DER
6. EINSTELLWERTE DER
7. SONSTIGE EINSTELLWERTE VON
8. RICHTWERTE BETRIEB DES
9. HAUPTBAUTEILE
10. SENSOREN
